In [1]:
import requests #api üzerinden döviz kurlarını çekmek için
import pandas as pd
import os
from datetime import datetime
from datetime import timedelta

# .env
API_KEY = "bcb099b37d8a46c5b639ddb86c8d9b64"


In [2]:
# belirli tarih için api'dan veri çekme. Url api websitesinden alındı.

def fetch_historical_rates(date_str, api_key):
    url = f"https://openexchangerates.org/api/historical/{date_str}.json?app_id={api_key}"
    response = requests.get(url)

    if response.status_code != 200:
        print(f"{date_str} veri alınamadı:", response.text)
        return None

    data = response.json()
    base = data["base"]
    rates = data["rates"]

    records = []
    for target_currency, rate in rates.items():
        records.append({
            "date": date_str,
            "base_currency": base,
            "target_currency": target_currency,
            "exchange_rate": rate
        })

    return pd.DataFrame(records)


In [3]:
# 2024 1 Ocak için ilk veriyi çekelim:
fetch_historical_rates("2024-01-01", API_KEY)


,date,base_currency,target_currency,exchange_rate
0,2024-01-01,USD,AED,3.672800
1,2024-01-01,USD,AFN,70.834276
2,2024-01-01,USD,ALL,93.793860
3,2024-01-01,USD,AMD,405.595377
4,2024-01-01,USD,ANG,1.807342
...,...,...,...,...
164,2024-01-01,USD,XPT,0.001005
165,2024-01-01,USD,YER,250.230000
166,2024-01-01,USD,ZAR,18.326048
167,2024-01-01,USD,ZMW,25.806890


In [4]:
# 2024 yılının tüm günleri için fonksiyonu çalıştıralım

def fetch_2024_data(api_key):
    start_date = datetime.strptime("2024-01-01", "%Y-%m-%d")
    end_date = datetime.strptime("2024-12-31", "%Y-%m-%d")

    all_data = []

    current_date = start_date
    while current_date <= end_date:
        date_str = current_date.strftime("%Y-%m-%d")
        df = fetch_historical_rates(date_str, api_key)
        if df is not None:
            all_data.append(df)
        current_date += timedelta(days=1)

    full_df = pd.concat(all_data, ignore_index=True)
    return full_df


In [5]:
# çekilen veride ilk 5 satır
df_2024 = fetch_2024_data(API_KEY)
df_2024.head()


,date,base_currency,target_currency,exchange_rate
0,2024-01-01,USD,AED,3.672800
1,2024-01-01,USD,AFN,70.834276
2,2024-01-01,USD,ALL,93.793860
3,2024-01-01,USD,AMD,405.595377
4,2024-01-01,USD,ANG,1.807342


In [6]:
df_2024

,date,base_currency,target_currency,exchange_rate
0,2024-01-01,USD,AED,3.672800
1,2024-01-01,USD,AFN,70.834276
2,2024-01-01,USD,ALL,93.793860
3,2024-01-01,USD,AMD,405.595377
4,2024-01-01,USD,ANG,1.807342
...,...,...,...,...
61849,2024-12-31,USD,XPT,0.001116
61850,2024-12-31,USD,YER,249.847249
61851,2024-12-31,USD,ZAR,18.899908
61852,2024-12-31,USD,ZMW,27.801554


In [7]:
# usd verilerinin csv dosyasına kaydedilmesi
df_2024.to_csv("usd_exchange_rates_2024.csv", index=False)


In [8]:
# Her gün aynı sayıda kayıt seti var mı kontrol
df_2024["date"].value_counts()


date
2024-01-01    169
2024-09-08    169
2024-09-06    169
2024-09-05    169
2024-09-04    169
             ... 
2024-04-30    169
2024-04-29    169
2024-04-28    169
2024-04-27    169
2024-12-31    169
Name: count, Length: 366, dtype: int64

In [11]:
def update_last_4_days(api_key, csv_path="last_4_days.csv"):
    today = datetime.strptime("2024-12-31", "%Y-%m-%d")  # Normalde datetime.today()
    updated_data = []

    for i in range(1, 5):
        date_str = (today - timedelta(days=i)).strftime("%Y-%m-%d")
        df = fetch_historical_rates(date_str, api_key)
        if df is not None:
            updated_data.append(df)

    new_df = pd.concat(updated_data, ignore_index=True)

    if os.path.exists(csv_path):
        existing_df = pd.read_csv(csv_path)
        existing_df["date"] = pd.to_datetime(existing_df["date"])
        new_df["date"] = pd.to_datetime(new_df["date"])

        key_cols = ["date", "target_currency"]
        existing_df = existing_df.merge(
            new_df[key_cols].drop_duplicates(),
            on=key_cols,
            how="left",
            indicator=True
        ).query('_merge == "left_only"').drop(columns="_merge")

        updated_df = pd.concat([existing_df, new_df], ignore_index=True)
    else:
        updated_df = new_df

    updated_df = updated_df.sort_values("date").reset_index(drop=True)
    updated_df.to_csv(csv_path, index=False)
    print("Son 4 gün verileri güncellendi ve duplicate'ler önlendi.")


In [12]:
update_last_4_days(API_KEY)


Son 4 gün verileri güncellendi ve duplicate'ler önlendi.
